# Day 25: The Halting Problem
[Link](http://adventofcode.com/2017/day/25)

## Puzzle input

Begin in state `A`.
Perform a diagnostic checksum after `12317297`steps.

### In state `A`:
If the current value is `0`:
- Write the value `1`.
- Move one slot to the `right`.
- Continue with state `B`.

If the current value is `1`:
- Write the value `0`.
- Move one slot to the `left`.
- Continue with state `D`.

### In state `B`:
If the current value is `0`:
- Write the value `1`.
- Move one slot to the `right`.
- Continue with state `C`.

If the current value is `1`:
- Write the value `0`.
- Move one slot to the `right`.
- Continue with state `F`.

### In state C:
If the current value is `0`:
- Write the value `1`.
- Move one slot to the `left`.
- Continue with state `C`.

If the current value is `1`:
- Write the value `1`.
- Move one slot to the `left`.
- Continue with state `A`.

### In state `D`:
If the current value is `0`:
- Write the value `0`.
- Move one slot to the `left`.
- Continue with state `E`.

If the current value is `1`:
- Write the value `1`.
- Move one slot to the `right`.
- Continue with state `A`.

### In state E:
If the current value is `0`:
- Write the value `1`.
- Move one slot to the `left`.
- Continue with state `A`.

If the current value is `1`:
- Write the value `0`.
- Move one slot to the `right`.
- Continue with state `B`.

### In state F:
If the current value is `0`:
- Write the value `0`.
- Move one slot to the `right`.
- Continue with state `C`.

If the current value is `1`:
- Write the value `0`.
- Move one slot to the `right`.
- Continue with state `E`.

## Turing Machine class

In [1]:
class TuringMachine:
    def __init__(self):
        self.pos_tape, self.neg_tape, self.position, self.state = [False], [], 0, self.state_a

    def tape(self):
        return self.pos_tape if self.position >= 0 else self.neg_tape

    def index(self):
        return self.position if self.position >= 0  else -1 - self.position

    def read(self):
        return self.tape()[self.index()]

    def write(self, value):
        self.tape()[self.index()] = value

    def move_r(self):
        self.position += 1
        self.init_slot()

    def move_l(self):
        self.position -= 1
        self.init_slot()

    def init_slot(self):
        if len(self.tape()) <= self.index():
            tape_ = self.tape()
            tape_ += [False] * (self.index() + 1 - len(self.tape()))

    def checksum(self):
        return sum(self.pos_tape) + sum(self.neg_tape)

    def run(self, steps=1, debug=False):
        for i in range(steps):
            self.state()
            if debug:
                print(str(self))

    def __str__(self):
        tape = []
        pos_tape = self.position >= 0
        idx = self.index()
        for i in range(len(self.neg_tape)-1, -1, -1):
            border_l = '[' if not pos_tape and i == idx else ' '
            border_r = ']' if not pos_tape and i == idx else ' '
            tape.append(f'{border_l}{int(self.neg_tape[i])}{border_r}')
        for i in range(len(self.pos_tape)):
            border_l = '[' if pos_tape and i == idx else ' '
            border_r = ']' if pos_tape and i == idx else ' '
            tape.append(f'{border_l}{int(self.pos_tape[i])}{border_r}')

        return f'Checksum: {self.checksum()} {self.state.__name__} {"".join(tape)}'

    def state_a(self):
        if self.read():
            self.write(False)
            self.move_l()
            self.state = self.state_d
        else:
            self.write(True)
            self.move_r()
            self.state = self.state_b

    def state_b(self):
        if self.read():
            self.write(False)
            self.move_r()
            self.state = self.state_f
        else:
            self.write(True)
            self.move_r()
            self.state = self.state_c

    def state_c(self):
        if self.read():
            self.write(True)
            self.move_l()
            self.state = self.state_a
        else:
            self.write(True)
            self.move_l()
            self.state = self.state_c

    def state_d(self):
        if self.read():
            self.write(True)
            self.move_r()
            self.state = self.state_a
        else:
            self.write(False)
            self.move_l()
            self.state = self.state_e

    def state_e(self):
        if self.read():
            self.write(False)
            self.move_r()
            self.state = self.state_b
        else:
            self.write(True)
            self.move_l()
            self.state = self.state_a

    def state_f(self):
        if self.read():
            self.write(False)
            self.move_r()
            self.state = self.state_e
        else:
            self.write(False)
            self.move_r()
            self.state = self.state_c

In [2]:
test_machine = TuringMachine()
test_steps = 10
test_machine.run(test_steps, True)
print(str(test_machine))

Checksum: 1 state_b  1 [0]
Checksum: 2 state_c  1  1 [0]
Checksum: 3 state_c  1 [1] 1 
Checksum: 3 state_a [1] 1  1 
Checksum: 2 state_d [0] 0  1  1 
Checksum: 2 state_e [0] 0  0  1  1 
Checksum: 3 state_a [0] 1  0  0  1  1 
Checksum: 4 state_b  1 [1] 0  0  1  1 
Checksum: 3 state_f  1  0 [0] 0  1  1 
Checksum: 3 state_c  1  0  0 [0] 1  1 
Checksum: 3 state_c  1  0  0 [0] 1  1 


In [3]:
machine = TuringMachine()
steps = 12317297
machine.run(steps)
print(f'Answer to part 1: the checksum after {steps} steps is {machine.checksum()}')  # 4230

Answer to part 1: the checksum after 12317297 steps is 4230
